In [3]:
import os
import re
import pandas as pd
import numpy as np
import datetime

In [2]:
# def scan_dir(rootpath):
#     # Сканирование директории в поисках файлов в формате SBIG ST-6
#     for dirpath, _, filenames in os.walk(rootpath):
#         for fn in filter(lambda x: "SBIG" in x, filenames):
#             yield os.path.join(dirpath, fn)

# def sbig_read_headers(fd):
#     # Чтение метаданных из SBIG файла
#     for line in map(str.strip, fd):
#         if "End" in line:
#             return
#         m = re.search("^([^=]*) = (.*)$", line)
#         if m:
#             yield m.group(1), m.group(2)

# def scan_sbig_files(rootpath):
#     # Рекурсивный поиск файлов и чтение метаданных
#     for fn in scan_dir(rootpath):
#         headers = {}
#         with open(fn, mode="rt", errors="ignore", newline="\n") as fd:
#             for k, v in sbig_read_headers(fd):
#                 headers[k] = v
#         yield fn, headers

In [3]:
# rows = []

# for fn, headers in scan_sbig_files("/s/science/ss433"):
#     d = headers
#     d["FilePath"] = fn
#     rows.append(d)

# df = pd.DataFrame(rows)
# df.to_csv("data/ss433.catalog")

In [14]:
df = pd.read_csv("data/ss433.catalog")

df = df[["FilePath", "Date", "Time", "Exposure", "Temperature"]]
df["File"] = [ x[x.rfind("/") + 1:] for x in df["FilePath"] ]

df["DateTime"] = df["Date"] + " " + df["Time"]
df["DateTime"] = pd.to_datetime(df["DateTime"], format='%m/%d/%y %H:%M:%S')

df = df[["FilePath", "File", "DateTime", "Exposure", "Temperature"]]
df = df.loc[lambda x: x["FilePath"].str.contains("ss433[^/]*.SBIG"), :]
df

# df["Exposure"] = pd.to_numeric(df["Exposure"])
# df["Temperature"] = pd.to_numeric(df["Temperature"])
# #df["Exposure"] = df["Exposure"].astype(np.float32)
# display(df.dtypes)

#display(df.style.set_properties(subset=["FilePath", "File"], **{'text-align': 'left'}))

,FilePath,File,DateTime,Exposure,Temperature
124,/s/science/ss433/ss433-16/0610/ss433-b.SBIG,ss433-b.SBIG,2016-06-10 23:23:57,180000,-10.20
132,/s/science/ss433/ss433-16/0610/ss433-801.SBIG,ss433-801.SBIG,2016-06-11 00:04:40,180000,-9.79
333,/s/science/ss433/ss433-16/0730/ss433r.SBIG,ss433r.SBIG,2016-07-30 19:48:59,180000,-4.82
336,/s/science/ss433/ss433-16/0730/ss433b.SBIG,ss433b.SBIG,2016-07-30 19:15:09,180000,-4.82
337,/s/science/ss433/ss433-16/0730/ss433rx.SBIG,ss433rx.SBIG,2016-07-30 19:48:59,180000,-4.82
...,...,...,...,...,...
1365,/s/science/ss433/ss433-14/1026/ss433ba.SBIG,ss433ba.SBIG,2014-10-26 15:47:27,180000,-22.20
1366,/s/science/ss433/ss433-14/1026/ss433r.SBIG,ss433r.SBIG,2014-10-26 16:22:57,180000,-22.20
1367,/s/science/ss433/ss433-14/1026/ss433b.SBIG,ss433b.SBIG,2014-10-26 15:26:19,60000,-22.20
1370,/s/science/ss433/ss433-14/0502/ss433.SBIG,ss433.SBIG,2014-05-03 02:07:29,60000,-15.23


In [5]:
df.groupby("Exposure").count()

,FilePath,File,DateTime,Temperature
Exposure,,,,
30000,1,1,1,1
60000,64,64,64,64
90000,2,2,2,2
180000,138,138,138,138


In [6]:
df.groupby(["Temperature", "Exposure"]).count()

FilePath  File  DateTime
Temperature Exposure                          
-25.40      60000            2     2         2
            180000           2     2         2
-23.10      180000           1     1         1
-22.20      60000            2     2         2
            180000           4     4         4
-20.42      60000            2     2         2
            180000           8     8         8
-17.37      60000            1     1         1
            180000           8     8         8
-15.23      60000           10    10        10
            90000            1     1         1
            180000          11    11        11
-14.81      180000           2     2         2
-13.12      60000            5     5         5
            180000          10    10        10
-12.71      60000            1     1         1
            180000           1     1         1
-10.20      30000            1     1         1
            60000           25    25        25
            180000          33    33        33
-9.79       60000            1     1         1
            180000           3     3         3
-7.30       60000           12    12        12
            90000            1     1         1
            180000          35    35        35
-6.89       180000           4     4         4
-5.23       60000            3     3         3
            180000          10    10        10
-4.82       180000           4     4         4
-2.32       180000           2     2         2

In [66]:
def temp(d):
    if d < -22.5:
        return -25
    elif d < -17.5:
        return -20
    elif d < -12.5:
        return -15
    elif d < -7.5:
        return -10
    elif d < -2.5:
        return -5
    elif d < 2.5:
        return 0
    elif d < 7.5:
        return 5
    return d

df["TempGroup"] = df["Temperature"].map(temp)
display(df.head(3))
df.groupby(["TempGroup", "Exposure"]).count()


,FilePath,File,DateTime,Exposure,Temperature,TempGroup,TempGroup2
124,/s/science/ss433/ss433-16/0610/ss433-b.SBIG,ss433-b.SBIG,2016-06-10 23:23:57,180000,-10.20,-10,-10.0
132,/s/science/ss433/ss433-16/0610/ss433-801.SBIG,ss433-801.SBIG,2016-06-11 00:04:40,180000,-9.79,-10,-10.0
333,/s/science/ss433/ss433-16/0730/ss433r.SBIG,ss433r.SBIG,2016-07-30 19:48:59,180000,-4.82,-5,-5.0


FilePath  File  DateTime  Temperature  TempGroup2
TempGroup Exposure                                                   
-25       60000            2     2         2            2           2
          180000           3     3         3            3           3
-20       60000            4     4         4            4           4
          180000          12    12        12           12          12
-15       60000           17    17        17           17          17
          90000            1     1         1            1           1
          180000          32    32        32           32          32
-10       30000            1     1         1            1           1
          60000           26    26        26           26          26
          180000          36    36        36           36          36
-5        60000           15    15        15           15          15
          90000            1     1         1            1           1
          180000          53    53        53           53          53
 0        180000           2     2         2            2           2

In [81]:
df["TempGroup2"] = df["Temperature"].map(lambda d: 5 * (((d - 2.5) // 5 + 1) if d < 0 else ((d + 2.5) // 5)))
display(df["TempGroup"].compare(df["TempGroup2"]))

,self,other
